In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import warnings
from openpyxl.styles import Alignment 
warnings.filterwarnings('ignore')

# Exports aggregator

## Importing data

In [19]:
exports_folder = "data/"
csv_file_name = 'Combined - DEC23.csv'
xlsx_file_name = 'CombinedFiltered - DEC23.xlsx'

exports = pd.concat([pd.read_csv(os.path.join(exports_folder, 'JUN23.csv'), sep=';', encoding='utf-8', decimal=','),
                     pd.read_csv(os.path.join(exports_folder, 'DEC23.csv'), sep=';', encoding='utf-8', decimal=',')])

countries_data = pd.read_excel("FM Quick Data.xlsx", sheet_name='Countries')
division_levels = pd.read_excel("FM Quick Data.xlsx", sheet_name='Division Levels')

exports.shape

(9588, 89)

In [20]:
def check_date(row):
    if row['Rgn'] == 'EU':
        return row['Date'] > last_date_eu
    elif row['Rgn'] == 'RW':
        return row['Date'] > last_date_rw
    else:
        return False 
    
season_ends = exports[exports['Half'] == 'End']
last_date_eu = season_ends[season_ends['Rgn'] == 'EU']['Date'].max()
last_date_rw = season_ends[season_ends['Rgn'] == 'RW']['Date'].max()

season_mids = exports[exports['Half'] == 'Mid']
season_mids['Dummy'] = season_mids.apply(check_date, axis=1)
season_mids = season_mids[season_mids['Dummy']]
season_mids.drop(columns=['Dummy'], inplace=True)

all_seasons = pd.concat([season_ends, season_mids])
all_seasons['Av_Rat_Total'] = all_seasons['Av_Rat'] * all_seasons['Apps']
all_seasons['NP-xG/Shot_Total'] = all_seasons['NP-xG/Shot'] * all_seasons['Apps']
all_seasons['Sprints'] = (all_seasons['Sprints/90'] * (all_seasons['Mins'] / 90)).round(0)
all_seasons['Mins_t'] = all_seasons['Mins']

agg_scheme = {
    'Rgn': 'last',
    'Half': 'last',
    'Date': 'last',
    'EU': 'last',
    'Nat': 'last',
    'Interested': 'last',
    'P1': 'last',
    'P2': 'last',
    'Rt1': 'last',
    'Pt1': 'last',
    'RtG': 'last',
    'PtG': 'last',
    'Position': 'last',
    'Age': 'last',
    'Name': 'last',
    'Club': 'last',
    'Division': 'last',
    'Mins': 'last',
    'Based': 'last',
    'GK%': 'last',
    'DFg%': 'last',
    'DFa%': 'last',
    'xGP/90': 'last',
    'CS/90': 'last',
    'Gls_Conc/90': 'last',
    'Hdrs_W/90': 'last',
    'Hd%': 'last',
    'Blocks/90': 'last',
    'Intcp/90': 'last',
    'Clear/90': 'last',
    'Press_C/90': 'last',
    'PAS%': 'last',
    'OP-KP/90': 'last',
    'CCC/90': 'last',
    'Pr_Pas/90': 'last',
    'xA/90': 'last',
    'Pas_C/90': 'last',
    'Ps%': 'last',
    'DRB%': 'last',
    'ST%': 'last',
    'Drb/90': 'last',
    'Cr_C/90': 'last',
    'Cr%': 'last',
    'NP-xG/90': 'last',
    'Gls/90': 'last',
    'NP-xG/Shot': 'last',
    'Sht_T/90': 'last',
    'Sprints/90': 'last',
    'Fls_Ag/90': 'last',
    'Value': 'last',
    'Wage': 'last',
    'Expires': 'last',
    'Det': 'last',
    'Prof': 'last',
    'Amb': 'last',
    'Acc': 'last',
    'Pac': 'last',
    'Height': 'last',
    'Personality': 'last',
    'Apps': 'sum',
    'Starts': 'sum',
    'Gls': 'sum',
    'Ast': 'sum',
    'PoM': 'sum',
    'Mins_t': 'sum',
    'xGP': 'sum',
    'CS': 'sum',
    'Gls_Conc': 'sum',
    'Blocks': 'sum',
    'Intcp': 'sum',
    'Clear': 'sum',
    'Press_C': 'sum',
    'Hdrs_A': 'sum',
    'Hdrs_W': 'sum',
    'OP-KP': 'sum',
    'CCC': 'sum',
    'Pr_Pas': 'sum',
    'xA': 'sum',
    'Pas_A': 'sum',
    'Pas_C': 'sum',
    'Cr_A': 'sum',
    'Cr_C': 'sum',
    'Drb': 'sum',
    'Fls_Ag': 'sum',
    'xG': 'sum',
    'NP-xG': 'sum',
    'Sht': 'sum',
    'Sht_T': 'sum',
    'Av_Rat_Total': 'sum',
    'NP-xG/Shot_Total': 'sum',
    'Sprints': 'sum'
}

agg = all_seasons.groupby('UID').agg(agg_scheme).reset_index()
agg['Av_Rat'] = (agg['Av_Rat_Total'] / agg['Apps']).round(2)
agg['NP-xG/Shot_t'] = (agg['NP-xG/Shot_Total'] / agg['Apps']).round(2)

agg['xGP/90_t'] = (agg['xGP'] / (agg['Mins_t'] / 90)).round(2)
agg['CS/90_t'] = (agg['CS'] / (agg['Mins_t'] / 90)).round(2)
agg['Gls_Conc/90_t'] = (agg['Gls_Conc'] / (agg['Mins_t'] / 90)).round(2)

agg['Blocks/90_t'] = (agg['Blocks'] / (agg['Mins_t'] / 90)).round(2)
agg['Intcp/90_t'] = (agg['Intcp'] / (agg['Mins_t'] / 90)).round(2)
agg['Clear/90_t'] = (agg['Clear'] / (agg['Mins_t'] / 90)).round(2)
agg['Press_C/90_t'] = (agg['Press_C'] / (agg['Mins_t'] / 90)).round(2)
agg['Hdrs_W/90_t'] = (agg['Hdrs_W'] / (agg['Mins_t'] / 90)).round(2)
agg['Hd%_t'] = (agg['Hdrs_W'] / (agg['Hdrs_A'] * 100)).round(0).fillna(0)

agg['OP-KP/90_t'] = (agg['OP-KP'] / (agg['Mins_t'] / 90)).round(2)
agg['CCC/90_t'] = (agg['CCC'] / (agg['Mins_t'] / 90)).round(2)
agg['Pr_Pas/90_t'] = (agg['Pr_Pas'] / (agg['Mins_t'] / 90)).round(2)
agg['xA/90_t'] = (agg['xA'] / (agg['Mins_t'] / 90)).round(2)
agg['Pas_C/90_t'] = (agg['Pas_C'] / (agg['Mins_t'] / 90)).round(2)
agg['Ps%_t'] = (agg['Pas_C'] / (agg['Pas_A'] * 100)).round(0).fillna(0)

agg['Drb/90_t'] = (agg['Drb'] / (agg['Mins_t'] / 90)).round(2)
agg['Spr/90_t'] = (agg['Sprints'] / (agg['Mins_t'] / 90)).round(2)
agg['Cr_C/90_t'] = (agg['Cr_C'] / (agg['Mins_t'] / 90)).round(2)
agg['Cr%_t'] = (agg['Cr_C'] / (agg['Cr_A'] * 100)).round(0).fillna(0)
agg['Fls_Ag/90_t'] = (agg['Fls_Ag'] / (agg['Mins_t'] / 90)).round(2)

agg['NP-xG/90_t'] = (agg['NP-xG'] / (agg['Mins_t'] / 90)).round(2)
agg['Gls/90_t'] = (agg['Gls'] / (agg['Mins_t'] / 90)).round(2)
agg['Sht_T/90_t'] = (agg['Sht_T'] / (agg['Mins_t'] / 90)).round(2)
agg['Shots_Gls%_t'] = (agg['Gls'] / (agg['Sht'] * 100)).round(0).fillna(0)

agg['GK%t'] = (agg['xGP/90_t'] + agg['CS/90_t'] - agg['Gls_Conc/90_t'] + (agg['Av_Rat'] / 10)).round(5)
agg['DFg%t'] = (agg['Blocks/90_t'] + agg['Intcp/90_t'] + agg['Clear/90_t'] + agg['Press_C/90_t']).round(5)
agg['DFa%t'] = (agg['Blocks/90_t'] + agg['Intcp/90_t'] + agg['Clear/90_t'] + agg['Press_C/90_t'] + agg['Hdrs_W/90_t'] + (agg['Hd%_t'] / 100)).round(5)
agg['PAS%t'] = (agg['OP-KP/90_t'] + agg['CCC/90_t'] + agg['Pr_Pas/90_t'] + agg['xA/90_t'] + (agg['Ps%_t']) / 100).round(5)
agg['DRB%t'] = (agg['Drb/90_t'] + agg['Cr_C/90_t'] + (agg['Cr%_t'] / 100) + agg['Fls_Ag/90_t'] + (agg['Spr/90_t'] / 9)).round(5)
agg['ST%t'] = (agg['NP-xG/90_t'] + agg['Gls/90_t'] + agg['Sht_T/90_t'] + (agg['Shots_Gls%_t'] / 100) + agg['NP-xG/Shot_t']).round(5)

# Define the desired column order
column_order = [
    'Rgn', 'Half', 'Date', 'UID', 'EU', 'Nat', 'Interested', 'P1', 'P2',
    'Rt1', 'Pt1', 'RtG', 'PtG', 'Position', 'Age', 'Name', 'Av_Rat', 'Club',
    'Division', 'Based', 'Apps', 'Starts', 'Gls', 'Ast', 'PoM', 'Mins',
    'GK%', 'DFg%', 'DFa%', 'xGP/90', 'CS/90', 'Gls_Conc/90', 'Hdrs_W/90', 'Hd%',
    'Blocks/90', 'Intcp/90', 'Clear/90', 'Press_C/90', 'PAS%', 'OP-KP/90', 'CCC/90',
    'Pr_Pas/90', 'xA/90', 'Pas_C/90', 'Ps%', 'DRB%', 'ST%', 'Drb/90', 'Cr_C/90',
    'Cr%', 'NP-xG/90', 'Gls/90', 'NP-xG/Shot', 'Sht_T/90', 'Sprints/90', 'Fls_Ag/90',
    'Value', 'Wage', 'Expires', 'Det', 'Prof', 'Amb', 'Acc', 'Pac', 'Height',
    'Personality', 'GK%t', 'DFg%t', 'DFa%t', 'PAS%t', 'DRB%t', 'ST%t', 'Mins_t']

# Reorder the columns in the 'agg' DataFrame
agg = agg[column_order]

## Exporting files

### CSV file

In [21]:
folder_agg = "data/"
excel_table = agg.copy()

csv_file_path = os.path.join(folder_agg, csv_file_name)
excel_table.to_csv(csv_file_path, index=False, encoding='utf-8', sep=';', decimal=',')

### XLSX filtered file

In [22]:
folder_agg = "data/"

interest = 1
val = 5000000
age = 34
mins = 1500
wage = float('inf')

excel_table = agg.copy()
excel_table_copy = excel_table[(excel_table['Mins_t'] >= mins) &
          (excel_table['Interested'] == 1) &                               
          (excel_table['Value'] <= val) &
          (excel_table['Wage'] < wage) &
          (excel_table['Age'] <= age)]

xlsx_file_path = os.path.join(folder_agg, xlsx_file_name)
excel_table_copy.to_excel(xlsx_file_path, index=False)

### Update columns

In [4]:
folder = "D:/Documents/Sports Interactive/Football Manager 2023/exports/Portugal/Jun 24"
update = pd.read_html(os.path.join(folder, 'update.html'), encoding="UTF-8", thousands=".", decimal=",")[0]

In [5]:
att = update

def count_hyphens(row, position):
    columns_to_count = []
    if position != "GK":
        columns_to_count = ['Det', 'Prof', 'Amb', 'Acc', 'Pac', 'Agi', 'Bal',
    'Sta', 'Ant', 'Cnt', 'Wor', 'Cro', 'Dri', 'Fin', 'Fir',
    'Jum', 'Mar', 'Pas', 'Str', 'Tck', 'Tec', 'Vis', 'Agg',
    'Bra', 'Cmp', 'Cor', 'Dec', 'Fla', 'Fre', 'Hea', 'Ldr',
    'Lon', 'L Th', 'Nat', 'OtB', 'Pen', 'Pos', 'Tea']
    else:
        columns_to_count = ['Det', 'Prof', 'Amb', 'Acc', 'Pac', 'Agi', 'Bal',
    'Sta', 'Ant', 'Cnt', 'Wor', 'Cro', 'Dri', 'Fin', 'Fir',
    'Jum', 'Mar', 'Pas', 'Str', 'Tck', 'Tec', 'Vis', 'Aer',
    'Ref', 'Han', '1v1', 'Agg', 'Bra', 'Cmd', 'Com', 'Cmp',
    'Cor', 'Dec', 'Ecc', 'Fla', 'Fre', 'Hea', 'Kic', 'Ldr',
    'Lon', 'L Th', 'Nat', 'OtB', 'Pen', 'Pos', 'Pun',
    'TRO', 'Tea', 'Thr']

    count = sum([1 for col in columns_to_count if "-" in str(row[col])])
    return count

gk_attributes = ['Aer', 'Ref', 'Han', '1v1', 'Ecc', 'Kic', 'Cmd', 'Com', 'Pun', 'TRO', 'Thr']

att_gk = att[att['Position'] == 'GK']
att_gk['Count'] = att_gk.apply(lambda row: count_hyphens(row, row['Position']), axis=1)
#att_gk = att_gk[['UID', 'Position', 'Name', 'Count']]

att_all = att[att['Position'] != 'GK']
att_all.drop(columns=gk_attributes, inplace=True)
att_all['Count'] = att_all.apply(lambda row: count_hyphens(row, row['Position']), axis=1)
#att_all = att_all[['UID', 'Position', 'Name', 'Count']]

att = pd.concat([att_all, att_gk], ignore_index=True)
att = att[att['Count'] == 0]
att = att[['UID', 'Personality', 'Name', 'Club', 'Based', 'AP', 'Wage', 'Det', 'Prof', 'Amb', 'Acc', 'Pac']]
att.rename(columns={'AP': 'Value'}, inplace=True)
att['Division'] = att['Based'].str.extract(r'\((.*?)\)')
att['Based'] = att['Based'].str.extract(r'^([^(]+)')
att['Based'] = att['Based'].str.strip()
att['Wage'] = att['Wage'].str.replace('.', '').str[2:-4].fillna(0).replace('', 0).astype(int)

if not (att['Value'] == 0).all():
    att['Aux1'] = att['Value'].str[-1]
    possible_values_aux = [(att['Aux1'] == '0'), (att['Aux1'] == 'K'), (att['Aux1'] == 'M')]
    replacements_aux = [0, 1000, 1000000]
    att['Aux2'] = np.select(possible_values_aux, replacements_aux, default=0)
    att['Aux3'] = att['Value'].str[2:]
    att['Aux4'] = att['Aux3'].apply(lambda x: 0 if x == '0' else x[:-1])
    att['Aux4'] = att['Aux4'].str.replace(',', '.').fillna(0).astype(float)
    att['Value'] = att['Aux4'] * att['Aux2']
    att['Value'] = att['Value'].astype(int)
    att = att.drop(columns=['Aux1', 'Aux2', 'Aux3', 'Aux4'], axis=1)

known = att['UID'].tolist()

# Joining aggregated table and new updates from HTML
total_data = agg.merge(att[['UID', 'Personality', 'Name', 'Club', 'Based', 'Value',
                            'Wage', 'Det', 'Prof', 'Amb', 'Acc', 'Pac', 'Division']], on='UID', how='left')

no_updates = total_data[total_data['Based_y'].isna()]
updates = total_data[total_data['Based_y'].notna()]

# Updating the players with available data
column_pairs = ['Personality', 'Name', 'Club', 'Based', 'Value', 'Wage', 'Det', 'Prof', 'Amb', 'Acc', 'Pac', 'Division']

for column in column_pairs:
    updates[column + '_x'] = updates[column + '_y']

# Creating the final dataframe
merge = pd.concat([no_updates, updates])
merge.drop(columns=[column + '_y' for column in column_pairs], inplace=True)
merge.rename(columns={column + '_x': column for column in column_pairs}, inplace=True)

for index, row in merge.iterrows():
    if row['UID'] in known:
        # If 'UID' is in the 'known' list, set Rt1 = RtG
        merge.at[index, 'Rt1'] = row['RtG']
        merge.at[index, 'Pt1'] = row['PtG']
    else:
        # If 'UID' is not in the 'known' list, leave Rt1 unchanged
        merge.at[index, 'Rt1'] = row['Rt1']
        merge.at[index, 'Pt1'] = row['Pt1']

# Setting the scouting network

In [6]:
# Global variables
continents_metrics = ['South America', 'Europe']
n_position = 30
max_level = 2
n_af = 25
n_eu = 25
n_sa = 25
n_na = 10
n_as = 10
n_oc = 5

# Rating variables
age = 32
value = 800000
mins = 1500
rat = 0
pot = 0
interest = 1

# Potential variables
age_p = 21
value_p = 100000
mins_p = 1500
rat_p = 0
pot_p = 0
interest_p = 1

## Genie rating table

In [7]:
# Table using career stats
genie_rating_career = merge[["UID", "Nat", "Interested", "P1", "P2",
                       "RtG", "PtG", "Position", "Age", "Name",
                       "Av_Rat", "Club", "Division", "Apps",
                       "Starts", "Gls", "Ast", "Acc", "Pac",
                       "Value", "Mins_t", "GK%t", "DFg%t",
                       "DFa%t", "PAS%t", "DRB%t", "ST%t", "Based"]]

genie_rating_career = genie_rating_career[
    (genie_rating_career['RtG'] >= rat) &
    (genie_rating_career['PtG'] >= pot) &
    (genie_rating_career['Value'] <= value) &
    (genie_rating_career['Interested'] >= interest) &
    (genie_rating_career['Age'] <= age) &
    (genie_rating_career['Mins_t'] >= mins)]

genie_rating_career.rename(columns={'Interested': 'Int',
                              'Av_Rat': 'AvR',
                              'Apps': 'App',
                              'Starts': 'Str',
                              'Mins_t': 'Mins',
                              'GK%t': 'GK',
                              'DFg%t': 'DFg',
                              'DFa%t': 'DFa',
                              'PAS%t': 'PAS',
                              'DRB%t': 'DRB',
                              'ST%t': 'ST'}, inplace=True)

genie_rating_career['Value'] = genie_rating_career['Value'].apply(lambda x: f'€{int(x):,}')
genie_rating_career[['Acc', 'Pac']] = genie_rating_career[['Acc', 'Pac']].astype(int)
genie_rating_career[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']] = genie_rating_career[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']].astype(str)
genie_rating_career = genie_rating_career.sort_values('RtG', ascending=False)
genie_rating_career = genie_rating_career.merge(countries_data[['Based', 'Continent']], on='Based', how='left')
genie_rating_career = genie_rating_career.merge(division_levels[['Division', 'Lv']], on='Division', how='left')
genie_rating_career['Lv'] = genie_rating_career['Lv'].fillna(0).astype(int)
genie_rating_career.drop_duplicates(subset=['UID'], inplace=True)
genie_rating_career.drop('UID', axis=1, inplace=True)

# Table using last export stats
genie_rating_season = merge[["UID", "Nat", "Interested", "P1", "P2",
                       "RtG", "PtG", "Position", "Age", "Name",
                       "Av_Rat", "Club", "Division", "Apps",
                       "Starts", "Gls", "Ast", "Acc", "Pac",
                       "Value", "Mins", "GK%", "DFg%",
                       "DFa%", "PAS%", "DRB%", "ST%", "Based"]]

genie_rating_season = genie_rating_season[
    (genie_rating_season['RtG'] >= rat) &
    (genie_rating_season['PtG'] >= pot) &
    (genie_rating_season['Value'] <= value) &
    (genie_rating_season['Interested'] >= interest) &
    (genie_rating_season['Age'] <= age) &
    (genie_rating_season['Mins'] >= mins)]

genie_rating_season.rename(columns={'Interested': 'Int',
                                 'Av_Rat': 'AvR',
                                 'Apps': 'App',
                                 'Starts': 'Str',
                                 'GK%': 'GK',
                                 'DFg%': 'DFg',
                                 'DFa%': 'DFa',
                                 'PAS%': 'PAS',
                                 'DRB%': 'DRB',
                                 'ST%': 'ST'}, inplace=True)

genie_rating_season['Value'] = genie_rating_season['Value'].apply(lambda x: f'€{int(x):,}')
genie_rating_season[['Acc', 'Pac']] = genie_rating_season[['Acc', 'Pac']].astype(int)
genie_rating_season[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']] = genie_rating_season[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']].astype(str)
genie_rating_season = genie_rating_season.sort_values('RtG', ascending=False)
genie_rating_season = genie_rating_season.merge(countries_data[['Based', 'Continent']], on='Based', how='left')
genie_rating_season = genie_rating_season.merge(division_levels[['Division', 'Lv']], on='Division', how='left')
genie_rating_season['Lv'] = genie_rating_season['Lv'].fillna(0).astype(int)
genie_rating_season.drop_duplicates(subset=['UID'], inplace=True)
genie_rating_season.drop('UID', axis=1, inplace=True)

## Genie potential table

In [8]:
# Table using career stats
genie_potential_career = merge[["UID", "Nat", "Interested", "P1", "P2",
                          "RtG", "PtG", "Position", "Age", "Name",
                          "Av_Rat", "Club", "Division", "Apps",
                          "Starts", "Gls", "Ast", "Acc", "Pac",
                          "Value", "Mins_t", "GK%t", "DFg%t",
                          "DFa%t", "PAS%t", "DRB%t", "ST%t", "Based"]]

genie_potential_career = genie_potential_career[
    (genie_potential_career['RtG'] >= rat_p) &
    (genie_potential_career['PtG'] >= pot_p) &
    (genie_potential_career['Value'] <= value_p) &
    (genie_potential_career['Interested'] >= interest_p) &   
    (genie_potential_career['Age'] <= age_p) &
    (genie_potential_career['Mins_t'] >= mins_p)]

genie_potential_career.rename(columns={'Interested': 'Int',
                                 'Av_Rat': 'AvR',
                                 'Apps': 'App',
                                 'Starts': 'Str',
                                 'Mins_t': 'Mins',
                                 'GK%t': 'GK',
                                 'DFg%t': 'DFg',
                                 'DFa%t': 'DFa',
                                 'PAS%t': 'PAS',
                                 'DRB%t': 'DRB',
                                 'ST%t': 'ST'}, inplace=True)

genie_potential_career['Value'] = genie_potential_career['Value'].apply(lambda x: f'€{int(x):,}')
genie_potential_career[['Acc', 'Pac']] = genie_potential_career[['Acc', 'Pac']].astype(int)
genie_potential_career[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']] = genie_potential_career[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']].astype(str)
genie_potential_career = genie_potential_career.sort_values('PtG', ascending=False)
genie_potential_career = genie_potential_career.merge(countries_data[['Based', 'Continent']], on='Based', how='left')
genie_potential_career = genie_potential_career.merge(division_levels[['Division', 'Lv']], on='Division', how='left')
genie_potential_career['Lv'] = genie_potential_career['Lv'].fillna(0).astype(int)
genie_potential_career.drop_duplicates(subset=['UID'], inplace=True)
genie_potential_career.drop('UID', axis=1, inplace=True)

# Table using last export stats
genie_potential_season = merge[["UID", "Nat", "Interested", "P1", "P2",
                          "RtG", "PtG", "Position", "Age", "Name",
                          "Av_Rat", "Club", "Division", "Apps",
                          "Starts", "Gls", "Ast", "Acc", "Pac",
                          "Value", "Mins", "GK%", "DFg%",
                          "DFa%", "PAS%", "DRB%", "ST%", "Based"]]

genie_potential_season = genie_potential_season[
    (genie_potential_season['RtG'] >= rat_p) &
    (genie_potential_season['PtG'] >= pot_p) &
    (genie_potential_season['Value'] <= value_p) &
    (genie_potential_season['Interested'] >= interest_p) &  
    (genie_potential_season['Age'] <= age_p) &
    (genie_potential_season['Mins'] >= mins_p)]

genie_potential_season.rename(columns={'Interested': 'Int',
                                 'Av_Rat': 'AvR',
                                 'Apps': 'App',
                                 'Starts': 'Str',
                                 'GK%': 'GK',
                                 'DFg%': 'DFg',
                                 'DFa%': 'DFa',
                                 'PAS%': 'PAS',
                                 'DRB%': 'DRB',
                                 'ST%': 'ST'}, inplace=True)

genie_potential_season['Value'] = genie_potential_season['Value'].apply(lambda x: f'€{int(x):,}')
genie_potential_season[['Acc', 'Pac']] = genie_potential_season[['Acc', 'Pac']].astype(int)
genie_potential_season[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']] = genie_potential_season[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']].astype(str)
genie_potential_season = genie_potential_season.sort_values('PtG', ascending=False)
genie_potential_season = genie_potential_season.merge(countries_data[['Based', 'Continent']], on='Based', how='left')
genie_potential_season = genie_potential_season.merge(division_levels[['Division', 'Lv']], on='Division', how='left')
genie_potential_season['Lv'] = genie_potential_season['Lv'].fillna(0).astype(int)
genie_potential_season.drop_duplicates(subset=['UID'], inplace=True)
genie_potential_season.drop('UID', axis=1, inplace=True)

## FM rating table

In [9]:
# Table using career stats
rating_career = merge[["UID", "Nat", "Interested", "P1", "P2",
                       "Rt1", "Pt1", "Position", "Age", "Name",
                       "Av_Rat", "Club", "Division", "Apps",
                       "Starts", "Gls", "Ast", "Acc", "Pac",
                       "Value", "Mins_t", "GK%t", "DFg%t",
                       "DFa%t", "PAS%t", "DRB%t", "ST%t", "Based"]]

rating_career = rating_career[
    (rating_career['Rt1'] >= rat) &
    (rating_career['Pt1'] >= pot) &
    (rating_career['Value'] <= value) &
    (rating_career['Interested'] >= interest) &
    (rating_career['Age'] <= age) &
    (rating_career['Mins_t'] >= mins)]

rating_career.rename(columns={'Interested': 'Int',
                              'Av_Rat': 'AvR',
                              'Apps': 'App',
                              'Starts': 'Str',
                              'Mins_t': 'Mins',
                              'GK%t': 'GK',
                              'DFg%t': 'DFg',
                              'DFa%t': 'DFa',
                              'PAS%t': 'PAS',
                              'DRB%t': 'DRB',
                              'ST%t': 'ST'}, inplace=True)

rating_career['Value'] = rating_career['Value'].apply(lambda x: f'€{int(x):,}')
rating_career[['Acc', 'Pac']] = rating_career[['Acc', 'Pac']].astype(int)
rating_career[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']] = rating_career[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']].astype(str)
rating_career = rating_career.sort_values('Rt1', ascending=False)
rating_career = rating_career.merge(countries_data[['Based', 'Continent']], on='Based', how='left')
rating_career = rating_career.merge(division_levels[['Division', 'Lv']], on='Division', how='left')
rating_career['Lv'] = rating_career['Lv'].fillna(0).astype(int)
rating_career.drop_duplicates(subset=['UID'], inplace=True)
rating_career.drop('UID', axis=1, inplace=True)

# Table using last export stats
rating_season = merge[["UID", "Nat", "Interested", "P1", "P2",
                       "Rt1", "Pt1", "Position", "Age", "Name",
                       "Av_Rat", "Club", "Division", "Apps",
                       "Starts", "Gls", "Ast", "Acc", "Pac",
                       "Value", "Mins", "GK%", "DFg%",
                       "DFa%", "PAS%", "DRB%", "ST%", "Based"]]

rating_season = rating_season[
    (rating_season['Rt1'] >= rat) &
    (rating_season['Pt1'] >= pot) &
    (rating_season['Value'] <= value) &
    (rating_season['Interested'] >= interest) &
    (rating_season['Age'] <= age) &
    (rating_season['Mins'] >= mins)]

rating_season.rename(columns={'Interested': 'Int',
                                 'Av_Rat': 'AvR',
                                 'Apps': 'App',
                                 'Starts': 'Str',
                                 'GK%': 'GK',
                                 'DFg%': 'DFg',
                                 'DFa%': 'DFa',
                                 'PAS%': 'PAS',
                                 'DRB%': 'DRB',
                                 'ST%': 'ST'}, inplace=True)

rating_season['Value'] = rating_season['Value'].apply(lambda x: f'€{int(x):,}')
rating_season[['Acc', 'Pac']] = rating_season[['Acc', 'Pac']].astype(int)
rating_season[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']] = rating_season[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']].astype(str)
rating_season = rating_season.sort_values('Rt1', ascending=False)
rating_season = rating_season.merge(countries_data[['Based', 'Continent']], on='Based', how='left')
rating_season = rating_season.merge(division_levels[['Division', 'Lv']], on='Division', how='left')
rating_season['Lv'] = rating_season['Lv'].fillna(0).astype(int)
rating_season.drop_duplicates(subset=['UID'], inplace=True)
rating_season.drop('UID', axis=1, inplace=True)

## FM potential table

In [10]:
# Table using career stats
potential_career = merge[["UID", "Nat", "Interested", "P1", "P2",
                          "Rt1", "Pt1", "Position", "Age", "Name",
                          "Av_Rat", "Club", "Division", "Apps",
                          "Starts", "Gls", "Ast", "Acc", "Pac",
                          "Value", "Mins_t", "GK%t", "DFg%t",
                          "DFa%t", "PAS%t", "DRB%t", "ST%t", "Based"]]

potential_career = potential_career[
    (potential_career['Rt1'] >= rat_p) &
    (potential_career['Pt1'] >= pot_p) &
    (potential_career['Value'] <= value_p) &
    (potential_career['Interested'] >= interest_p) &
    (potential_career['Age'] <= age_p) &
    (potential_career['Mins_t'] >= mins_p)]

potential_career.rename(columns={'Interested': 'Int',
                                 'Av_Rat': 'AvR',
                                 'Apps': 'App',
                                 'Starts': 'Str',
                                 'Mins_t': 'Mins',
                                 'GK%t': 'GK',
                                 'DFg%t': 'DFg',
                                 'DFa%t': 'DFa',
                                 'PAS%t': 'PAS',
                                 'DRB%t': 'DRB',
                                 'ST%t': 'ST'}, inplace=True)

potential_career['Value'] = potential_career['Value'].apply(lambda x: f'€{int(x):,}')
potential_career[['Acc', 'Pac']] = potential_career[['Acc', 'Pac']].astype(int)
potential_career[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']] = potential_career[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']].astype(str)
potential_career = potential_career.sort_values('Pt1', ascending=False)
potential_career = potential_career.merge(countries_data[['Based', 'Continent']], on='Based', how='left')
potential_career = potential_career.merge(division_levels[['Division', 'Lv']], on='Division', how='left')
potential_career['Lv'] = potential_career['Lv'].fillna(0).astype(int)
potential_career.drop_duplicates(subset=['UID'], inplace=True)
potential_career.drop('UID', axis=1, inplace=True)

# Table using last export stats
potential_season = merge[["UID", "Nat", "Interested", "P1", "P2",
                          "Rt1", "Pt1", "Position", "Age", "Name",
                          "Av_Rat", "Club", "Division", "Apps",
                          "Starts", "Gls", "Ast", "Acc", "Pac",
                          "Value", "Mins", "GK%", "DFg%",
                          "DFa%", "PAS%", "DRB%", "ST%", "Based"]]

potential_season = potential_season[
    (potential_season['Rt1'] >= rat_p) &
    (potential_season['Pt1'] >= pot_p) &
    (potential_season['Value'] <= value_p) &
    (potential_season['Interested'] >= interest_p) &
    (potential_season['Age'] <= age_p) &
    (potential_season['Mins'] >= mins_p)]

potential_season.rename(columns={'Interested': 'Int',
                                 'Av_Rat': 'AvR',
                                 'Apps': 'App',
                                 'Starts': 'Str',
                                 'GK%': 'GK',
                                 'DFg%': 'DFg',
                                 'DFa%': 'DFa',
                                 'PAS%': 'PAS',
                                 'DRB%': 'DRB',
                                 'ST%': 'ST'}, inplace=True)

potential_season['Value'] = potential_season['Value'].apply(lambda x: f'€{int(x):,}')
potential_season[['Acc', 'Pac']] = potential_season[['Acc', 'Pac']].astype(int)
potential_season[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']] = potential_season[['App', 'Str', 'Gls', 'Ast', 'Value', 'Mins']].astype(str)
potential_season = potential_season.sort_values('Pt1', ascending=False)
potential_season = potential_season.merge(countries_data[['Based', 'Continent']], on='Based', how='left')
potential_season = potential_season.merge(division_levels[['Division', 'Lv']], on='Division', how='left')
potential_season['Lv'] = potential_season['Lv'].fillna(0).astype(int)
potential_season.drop_duplicates(subset=['UID'], inplace=True)
potential_season.drop('UID', axis=1, inplace=True)

# Scouting

In [35]:
# Deciding which version to use
scouting1 = rating_career # Ordered by FM Rating with career stats
scouting2 = rating_season # Ordered by FM Rating with last export stats
scouting3 = potential_career # Ordered by FM Potential with career stats
scouting4 = potential_season # Ordered by FM Potential with last export stats
scouting5 = genie_rating_career # Ordered by Genie Rating with career stats
scouting6 = genie_rating_season # Ordered by Genie Rating with last export stats
scouting7 = genie_potential_career # Ordered by Genie Potential with career stats
scouting8 = genie_potential_season # Ordered by Genie Potential with last export stats

scouting = scouting5

## Top N players in the world

In [36]:
scouting.head(10).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P1,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
0,ENG,1,LB,RB,80.9,85.7,D (RL),23,Brandon Williams,6.8,Man Utd,Premier League,68,65,3,5,1318,1320,"€43,000",5750,0.7,9.9,12.9,4.3,7.1,0.4,England,Europe,1
8,ITA,1,LB,LW,80.5,85.3,D/WB/M (L),25,Emanuele Valeri,6.8,Cremonese,Serie B,77,77,1,8,1117,816,"€275,000",6452,0.7,8.8,10.6,5.0,7.2,0.3,Italy,Europe,2
9,BRA,1,LB,RB,80.3,81.3,D (L),31,Renê,6.8,Internacional,Brasileirão Assaí,43,42,1,2,14,13,"€575,000",3552,0.7,9.0,11.4,6.8,4.9,0.2,Brazil,South America,1
10,POR,1,CM,DM,80.1,81.1,"DM, M/AM (C)",28,Samu,6.8,Vizela,Liga bwin,78,78,0,10,11,13,"€155,000",6755,0.7,7.8,9.4,5.8,3.1,0.3,Portugal,Europe,1
11,ITA,1,LB,RB,79.1,83.1,D/WB (L),24,Gianluca Frabotta,6.9,Juventus,Serie A,42,39,0,5,1217,814,"€675,000",3403,0.7,8.9,12.7,6.8,5.5,0.1,Italy,Europe,1
12,CZE,1,LB,CM,79.0,79.6,"D/WB (L), M (C)",32,Martin Frýdek,6.9,Luzern,Super League,65,65,1,4,0,0,"€130,000",5699,0.7,9.1,13.6,6.5,5.7,0.3,Switzerland,Europe,1
14,DEN,1,LB,LW,78.8,79.7,D (L),29,Casper Højer,7.0,Sparta Praha,1. ČFL,83,81,1,10,0,0,"€625,000",7121,0.7,8.9,12.3,7.5,6.4,0.1,Czech Republic,Europe,1
15,ITA,1,RB,LB,78.5,80.0,D/WB (R),31,Stefano Sabelli,6.8,Genoa,Serie A,69,67,0,4,1420,1217,"€350,000",5189,0.7,8.7,12.5,3.9,6.3,0.2,Italy,Europe,1
16,JPN,1,RW,LW,78.4,80.1,AM (RL),26,Takahiro Kunimoto,7.7,Johor DT,Liga Super,28,27,7,6,0,0,"€250,000",2324,0.8,2.2,2.6,1.7,3.5,1.6,Malaysia,Asia,1
17,ENG,1,RB,CB,78.2,78.7,D (R),32,Luke Ayling,6.8,Leeds,EFL Championship,38,26,0,4,712,1018,€0,2329,0.7,8.9,13.2,4.4,5.5,0.2,England,Europe,2


## Top N players by position

### Goalkeepers

In [37]:
pd.set_option('display.max_columns', 30)
gks = scouting[scouting['P1'] == 'GK']
gks.drop('P1', axis=1, inplace=True)
gks.drop('P2', axis=1, inplace=True)
gks.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
23,COM,1,77.4,78.2,GK,32,Ali Ahamada,6.9,Azam FC,Vodacom Premier League,47,47,0,0,0,0,"€81,000",3980,0.3,0.0,0.0,0.1,0.8,0.0,Tanzania,Africa,1
55,BUL,1,76.9,78.8,GK,26,Daniel Naumov,6.9,CSKA 1948,Prva liga,24,24,0,0,0,0,"€450,000",2160,0.3,0.0,0.0,0.0,1.2,0.0,Bulgaria,Europe,1
63,FRA,1,76.6,78.1,GK,28,Thomas Didillon,6.6,Cercle Brugge,Eerste klasse A,71,70,0,0,0,0,"€115,000",6217,-0.2,0.0,0.0,0.1,0.6,0.0,Belgium,Europe,1
122,DEN,1,75.9,76.6,GK,31,Frederik Rønnow,6.7,Union Berlin,Bundesliga,53,51,0,0,0,0,€0,4566,-0.4,0.0,0.0,0.1,0.3,0.0,Germany,Europe,1
160,GER,1,75.4,81.5,GK,24,Nico Mantl,6.7,RB Salzburg,Bundesliga,71,70,0,0,0,0,"€725,000",6404,0.2,0.0,0.0,0.0,0.4,0.0,Austria,Europe,1
165,SWE,1,75.4,76.8,GK,30,Jacob Rinne,6.8,Al-Fateh,Saudi League,29,29,0,0,0,0,"€170,000",2574,0.3,0.0,0.0,0.0,0.5,0.0,Saudi Arabia,Asia,1
196,URU,1,75.1,76.3,GK,26,Santiago Mele,6.9,Plaza Colonia,1ª División,40,40,0,0,0,0,"€750,000",3363,0.7,0.0,0.0,0.0,0.5,0.0,Uruguay,South America,1
212,GER,1,75.0,75.5,GK,30,Jannik Huth,6.8,SC Paderborn,3. Liga,75,75,0,0,0,0,"€425,000",6731,-0.2,0.0,0.0,0.1,0.5,0.0,Germany,Europe,3
228,COL,1,74.9,76.9,GK,24,Kevin Mier,6.9,Atlético Nacional,Primera A,85,84,0,0,712,813,"€700,000",7156,0.5,0.0,0.0,0.0,0.8,0.0,Colombia,South America,1
264,RUS,1,74.6,75.5,GK,32,Andrey Lunev,7.0,Qarabağ,Premyer Liqası,74,74,0,0,0,0,"€575,000",6251,0.7,0.0,0.0,0.0,0.5,0.0,Azerbaijan,Europe,1


### Center backs

In [38]:
pd.set_option('display.max_columns', 30)
cbs = scouting[scouting['P1'] == 'CB']
cbs.drop('P1', axis=1, inplace=True)
cbs.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
64,SUI,1,RB,76.6,81.7,D (C),22,Leonidas Stergiou,6.7,St. Gallen,Super League,42,42,0,3,0,0,"€800,000",3704,0.7,7.5,13.3,4.8,1.3,0.3,Switzerland,Europe,1
67,ALG,1,DM,76.6,78.7,D (C),24,Mohamed Tougai,7.0,ES Tunis,Ligue 1,70,70,3,0,0,0,"€275,000",5933,0.7,6.7,14.5,4.0,1.1,0.4,Tunisia,Africa,1
142,DEN,1,RB,75.6,80.3,D (C),24,Japhet Sery Larsen,6.6,Brann,Eliteserien,44,44,0,7,0,0,"€800,000",3810,0.7,6.9,13.8,4.5,1.3,0.2,Norway,Europe,1
153,DEN,1,RB,75.4,79.8,D (RC),27,Stefan Gartenmann,6.9,FC Midtjylland,Superliga,57,52,1,3,0,0,"€500,000",4847,0.7,7.3,14.1,4.7,2.0,0.3,Denmark,Europe,1
179,POL,1,RB,75.2,81.4,D (C),23,Maik Nawrocki,6.7,Celtic,cinch Premiership,41,41,0,2,0,0,"€800,000",3671,0.7,6.7,13.3,4.5,1.2,0.2,Scotland,Europe,1
195,SRB,1,RB,75.1,75.2,D (C),28,Vukašin Jovanović,6.8,Čukarički,SuperLiga,33,32,1,1,0,0,"€165,000",2619,0.7,7.1,14.9,4.4,1.6,0.3,Serbia,Europe,1
204,BRA,1,DM,75.1,75.0,D (C),29,Yuri,6.9,Neftçi,Premyer Liqası,49,49,1,1,0,0,"€750,000",4391,0.7,6.6,13.7,3.5,1.2,0.3,Azerbaijan,Europe,1
234,NED,1,RB,74.9,75.1,D (C),32,Jeffrey Gouweleeuw,6.8,Augsburg,2. Bundesliga,60,43,0,0,0,0,"€120,000",4293,0.7,7.0,13.0,4.0,1.3,0.3,Germany,Europe,2
240,BRA,1,DM,74.8,75.3,D (C),30,Léo Coelho,7.0,Peñarol,1ª División,39,39,3,1,0,0,"€725,000",3413,0.7,6.8,14.5,4.0,1.1,0.5,Uruguay,South America,1
279,CMR,1,DM,74.5,77.4,"DM, M (C)",28,Gaël Ondoua,6.7,Servette,Super League,78,44,2,8,0,0,"€575,000",4381,0.7,6.9,10.5,5.0,2.4,0.4,Switzerland,Europe,1


### Left backs

In [39]:
pd.set_option('display.max_columns', 30)
lbs = scouting[scouting['P1'] == 'LB']
lbs.drop('P1', axis=1, inplace=True)
lbs.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
0,ENG,1,RB,80.9,85.7,D (RL),23,Brandon Williams,6.8,Man Utd,Premier League,68,65,3,5,1318,1320,"€43,000",5750,0.7,9.9,12.9,4.3,7.1,0.4,England,Europe,1
8,ITA,1,LW,80.5,85.3,D/WB/M (L),25,Emanuele Valeri,6.8,Cremonese,Serie B,77,77,1,8,1117,816,"€275,000",6452,0.7,8.8,10.6,5.0,7.2,0.3,Italy,Europe,2
9,BRA,1,RB,80.3,81.3,D (L),31,Renê,6.8,Internacional,Brasileirão Assaí,43,42,1,2,14,13,"€575,000",3552,0.7,9.0,11.4,6.8,4.9,0.2,Brazil,South America,1
11,ITA,1,RB,79.1,83.1,D/WB (L),24,Gianluca Frabotta,6.9,Juventus,Serie A,42,39,0,5,1217,814,"€675,000",3403,0.7,8.9,12.7,6.8,5.5,0.1,Italy,Europe,1
12,CZE,1,CM,79.0,79.6,"D/WB (L), M (C)",32,Martin Frýdek,6.9,Luzern,Super League,65,65,1,4,0,0,"€130,000",5699,0.7,9.1,13.6,6.5,5.7,0.3,Switzerland,Europe,1
14,DEN,1,LW,78.8,79.7,D (L),29,Casper Højer,7.0,Sparta Praha,1. ČFL,83,81,1,10,0,0,"€625,000",7121,0.7,8.9,12.3,7.5,6.4,0.1,Czech Republic,Europe,1
20,ARG,1,RB,77.8,79.5,D (L),25,Facundo Castet,7.0,Sarmiento (Junín),Primera Nacional,48,48,2,2,0,0,"€650,000",4181,0.7,5.2,7.2,4.0,3.9,0.4,Argentina,South America,2
25,ISR,1,LW,77.4,77.7,D/WB (L),30,Sun Menachem,6.7,Maccabi Haifa,Ligat Ha`Al,65,36,0,3,0,0,"€500,000",3794,0.7,9.1,12.9,7.2,4.7,0.3,Israel,Europe,1
38,ECU,1,CB,77.1,78.0,D (LC),26,Luis Segovia,6.8,Botafogo,Brasileirão Assaí,20,18,0,0,13,13,"€375,000",1638,0.7,7.0,13.8,4.5,1.4,0.3,Brazil,South America,1
39,IRN,1,RB,77.0,78.6,D/WB (L),30,Milad Mohammadi,6.7,AEK,Super League,72,64,0,2,1117,816,"€190,000",5778,0.7,9.1,12.0,6.5,5.1,0.1,Greece,Europe,1


### Right backs

In [40]:
pd.set_option('display.max_columns', 30)
rbs = scouting[scouting['P1'] == 'RB']
rbs.drop('P1', axis=1, inplace=True)
rbs.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
15,ITA,1,LB,78.5,80.0,D/WB (R),31,Stefano Sabelli,6.8,Genoa,Serie A,69,67,0,4,1420,1217,"€350,000",5189,0.7,8.7,12.5,3.9,6.3,0.2,Italy,Europe,1
17,ENG,1,CB,78.2,78.7,D (R),32,Luke Ayling,6.8,Leeds,EFL Championship,38,26,0,4,712,1018,€0,2329,0.7,8.9,13.2,4.4,5.5,0.2,England,Europe,2
19,NED,1,LB,78.0,78.7,"D (RL), WB/AM (R)",30,Joshua Brenet,7.0,Twente,Eredivisie,104,102,13,7,1218,916,"€525,000",9257,0.7,7.9,11.5,4.9,6.7,0.8,Netherlands,Europe,1
22,BOL,1,LB,77.6,78.3,D (R),32,Diego Bejarano,7.0,Bolívar,División Profesional,43,40,0,6,0,0,"€6,500",3523,0.7,8.5,12.4,5.5,6.8,0.3,Bolivia,South America,1
26,COL,1,RW,77.3,81.6,D/WB (R),29,Walmer Pacheco,6.8,Junior,Primera A,72,70,0,4,1217,813,"€400,000",6179,0.7,8.8,11.7,5.1,6.6,0.4,Colombia,South America,1
35,BRA,1,LB,77.2,81.4,"D (RL), WB (R)",29,Zeca,6.9,Vitória,Brasileirão SportingBet Série B,60,60,4,9,14,13,"€115,000",5243,0.7,9.5,12.2,7.3,6.3,0.3,Brazil,South America,2
36,JPN,1,LB,77.2,78.7,D/WB (R),30,Sei Muroya,6.8,Hannover 96,Bundesliga,71,68,1,6,0,0,"€775,000",5753,0.7,8.4,10.0,4.3,7.7,0.2,Germany,Europe,1
41,ARG,1,LB,77.0,78.1,D/WB (R),27,Hernán de la Fuente,6.9,nan,nan,79,74,3,6,14,14,€0,6379,0.7,9.5,12.3,5.9,7.0,0.4,Argentina,South America,0
42,COL,1,LB,77.0,84.2,D (RLC),24,Anderson Arroyo,7.0,Liverpool,Premier League,43,42,2,1,1119,1016,€0,3749,0.7,8.9,12.3,4.6,6.0,0.5,England,Europe,1
50,ARG,1,RW,77.0,78.7,D/WB (R),25,Alex Vigo,6.8,River,Liga Profesional,28,27,1,1,0,0,"€800,000",2481,0.7,9.3,13.1,5.0,5.8,0.5,Argentina,South America,1


### Defensive mids

In [41]:
pd.set_option('display.max_columns', 30)
dms = scouting[scouting['P1'] == 'DM']
dms.drop('P1', axis=1, inplace=True)
dms.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
24,ARG,1,CM,77.4,79.0,"DM, M (C)",31,Gerónimo Poblete,6.9,Al-Wasl,UAE Pro League,50,50,0,11,1018,1016,"€775,000",4477,0.7,7.3,10.2,7.0,3.1,0.2,UAE,Asia,1
71,NOR,1,CM,76.6,79.0,DM,28,Herman Stengel,6.9,Strømsgodset,Eliteserien,32,32,0,5,0,0,"€95,000",2866,0.7,6.5,9.6,7.0,3.0,0.3,Norway,Europe,1
73,CHI,1,CM,76.5,77.1,M (C),32,Leonardo Gil,7.0,Colo Colo,Primera División,66,64,5,16,0,0,"€675,000",5167,0.7,7.2,9.1,5.5,4.2,0.6,Chile,South America,1
135,FRA,1,CM,75.6,79.0,"DM, M (C)",28,Vincent Koziello,6.7,Oostende,Eerste klasse A,68,67,2,7,1015,1017,"€550,000",4806,0.7,7.4,9.4,6.9,3.1,0.3,Belgium,Europe,1
145,ALG,1,CM,75.6,77.2,"DM, M (C)",30,Victor Lekhal,6.8,Umm-Salal,QSL,33,33,0,5,714,1217,"€800,000",2527,0.7,7.6,11.5,6.5,3.0,0.3,Qatar,Asia,1
173,BRA,1,CM,75.3,78.8,DM,24,Caio Alexandre,6.9,Portimonense,Liga bwin,51,47,2,4,12,12,€0,3581,0.7,7.1,9.1,6.9,2.8,0.5,Portugal,Europe,1
180,ARG,1,CM,75.2,76.1,"DM, M (C)",31,Adrián Arregui,6.8,Independiente Medellín,Primera A,37,35,2,5,1217,1015,"€300,000",2847,0.7,7.5,10.0,6.3,2.6,0.5,Colombia,South America,1
186,COL,1,CM,75.2,77.7,"DM, M (C)",27,Bryan Rovira,6.9,Univ. Católica,Primera División,37,36,2,7,0,0,"€16,500",2987,0.7,7.3,10.8,7.5,3.0,0.5,Chile,South America,1
192,BOL,1,RB,75.2,76.1,DM,30,Diego Wayar,6.8,The Strongest,División Profesional,56,30,1,2,0,0,"€235,000",2988,0.7,8.3,10.2,4.9,5.2,0.3,Bolivia,South America,1
222,MAR,1,CM,75.0,75.7,"DM, M (C)",32,Yahya Jabrane,7.0,Wydad Casablanca,Botola,88,85,4,18,0,0,"€675,000",6959,0.7,7.1,10.5,5.2,2.5,0.4,Morocco,Africa,1


### Central mids

In [42]:
pd.set_option('display.max_columns', 30)
cms = scouting[scouting['P1'] == 'CM']
cms.drop('P1', axis=1, inplace=True)
cms.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
10,POR,1,DM,80.1,81.1,"DM, M/AM (C)",28,Samu,6.8,Vizela,Liga bwin,78,78,0,10,11,13,"€155,000",6755,0.7,7.8,9.4,5.8,3.1,0.3,Portugal,Europe,1
18,NGA,1,DM,78.0,81.4,M/AM (C),26,Michael Folorunsho,6.8,Napoli,Serie A,41,37,3,3,1117,614,"€27,500",2977,0.7,7.5,11.4,5.9,3.3,0.6,Italy,Europe,1
21,ARG,1,DM,77.8,78.8,M (C),30,Leonel Miranda,6.9,Racing Club,Liga Profesional,37,30,1,4,0,0,"€300,000",2503,0.7,6.7,8.4,7.1,3.0,0.3,Argentina,South America,1
52,CRO,1,DM,76.9,77.6,"DM, M (C)",32,Filip Bradarić,7.2,Zrinjski,Premijer Liga BiH,33,33,5,12,0,0,"€525,000",2781,0.7,7.6,11.7,6.5,4.9,0.7,Bosnia & Herzegovina,Europe,1
103,BEL,1,DM,76.2,79.5,"DM, M (C)",30,Birger Verstraete,6.7,Antwerp,Eerste klasse A,89,39,3,6,1116,813,"€125,000",4401,0.7,6.6,9.4,5.1,3.1,0.3,Belgium,Europe,1
114,SCO,1,RW,76.0,79.4,M/AM (C),25,Kyle Magennis,6.8,Kilmarnock,cinch Premiership,68,67,11,3,0,0,"€650,000",4565,0.7,6.4,9.0,3.5,4.1,1.1,Scotland,Europe,1
117,ENG,1,RB,76.0,82.3,"DM, M (C)",24,Jamie Shackleton,6.7,Leeds,EFL Championship,67,66,1,6,1218,816,€0,5589,0.7,6.5,8.4,6.2,2.9,0.3,England,Europe,2
124,POR,1,DM,75.9,77.9,M/AM (C),31,Pedrinho,6.8,Ankaragücü,Süper Lig,57,32,5,4,1016,512,"€375,000",3098,0.7,6.2,8.3,4.5,3.1,1.0,Turkey,Europe,1
139,ECU,1,DM,75.6,77.5,"DM, M (C)",25,Jhonny Quiñónez,6.9,Aucas,Serie A de Ecuador,54,54,3,6,0,0,"€675,000",4215,0.7,7.1,10.3,7.2,2.8,0.4,Ecuador,South America,1
141,CMR,1,DM,75.6,78.2,"DM, M/AM (C)",27,Olivier Kemen,7.0,Kayserispor,Süper Lig,71,70,7,10,915,714,"€525,000",5921,0.7,7.2,9.9,4.3,3.1,0.9,Turkey,Europe,1


### Attacking mids

In [43]:
pd.set_option('display.max_columns', 30)
ams = scouting[scouting['P1'] == 'AM']
ams.drop('P1', axis=1, inplace=True)
ams.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
121,CZE,1,RW,75.9,78.0,AM (C),25,Ondřej Lingr,6.8,Slavia Praha,1. ČFL,60,25,10,4,0,0,"€550,000",2980,0.7,5.8,8.9,2.8,3.8,1.9,Czech Republic,Europe,1
502,ESP,1,LW,73.6,76.4,AM (C),30,Jorge Pombo,6.8,Racing,LaLiga 2,53,52,1,4,1018,814,"€165,000",3860,0.7,6.5,8.7,5.7,3.3,0.4,Spain,Europe,2
522,TUN,1,RW,73.5,74.7,AM (C),30,Saad Bguir,7.1,Abha,Saudi League,42,39,10,4,1119,915,"€700,000",2881,0.7,6.3,7.4,4.7,6.1,1.6,Saudi Arabia,Asia,1
640,ARG,1,CM,73.1,73.9,M/AM (C),32,Claudio Aquino,6.8,Vélez,Liga Profesional,35,29,2,7,1117,913,"€240,000",2099,0.7,8.3,9.9,5.8,5.3,0.4,Argentina,South America,1
835,ROU,1,CM,72.7,74.5,M/AM (C),27,Ronaldo Deaconu,7.1,Korona,1. Liga,64,64,21,21,1318,1015,"€85,000",5333,0.7,6.5,9.0,4.2,4.2,1.8,Poland,Europe,2
878,BOL,1,ST,72.6,73.3,"AM (C), ST (C)",27,Víctor Abrego,6.9,Bolívar,División Profesional,60,39,15,5,0,0,"€205,000",3496,0.7,4.6,8.4,1.9,5.2,2.3,Bolivia,South America,1
986,NOR,1,RW,72.3,76.7,AM (C),22,Herman Geelmuyden,6.8,Stabæk,Eliteserien,49,48,6,10,0,0,"€775,000",3680,0.7,5.9,8.3,3.2,4.0,1.0,Norway,Europe,1
1120,ARG,1,RW,72.1,77.1,AM (C),29,Tomás Martínez,6.7,Melgar,Liga 1,66,34,2,9,0,0,"€525,000",3363,0.7,6.3,7.9,4.2,3.6,0.7,Peru,South America,1
1344,ARG,1,CM,71.8,74.9,AM (C),28,Leonardo Heredia,6.8,Atl. Tucumán,Liga Profesional,54,31,6,8,0,0,"€675,000",3052,0.7,6.5,9.7,4.3,2.6,0.8,Argentina,South America,1
1475,VEN,1,CM,71.6,73.5,M/AM (C),27,Williams Lugo,6.8,Academia Puerto Cabello,Liga FUTVE,64,63,15,6,1419,1116,"€81,000",5028,0.7,6.2,8.6,2.5,4.1,1.2,Venezuela,South America,1


### Left wingers

In [44]:
pd.set_option('display.max_columns', 30)
lws = scouting[scouting['P1'] == 'LW']
lws.drop('P1', axis=1, inplace=True)
lws.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
53,GRE,1,RW,76.9,78.2,AM (RL),30,Giorgos Masouras,6.9,Olympiacos,Super League,79,60,9,5,715,1420,"€250,000",5195,0.7,7.4,9.5,3.1,6.2,1.2,Greece,Europe,1
102,VEN,1,RW,76.2,82.0,AM (RL),25,Freddy Vargas,7.2,Metropolitanos,Liga FUTVE,58,58,15,6,1015,1017,"€155,000",4852,0.7,6.6,9.5,3.8,7.3,1.5,Venezuela,South America,1
130,COL,1,RW,75.7,76.9,"M (L), AM (RL)",30,Alex Castro,6.9,Águilas Doradas,Primera A,65,61,5,8,916,1419,"€250,000",5254,0.7,8.4,10.8,4.7,7.4,0.5,Colombia,South America,1
137,FRA,1,RW,75.6,75.4,AM (RL),32,Abdellah Zoubir,6.9,Qarabağ,Premyer Liqası,70,69,8,11,0,0,"€38,000",5995,0.7,7.1,9.2,3.6,6.9,0.9,Azerbaijan,Europe,1
140,COL,1,RW,75.6,79.0,"M (L), AM (RL)",28,Omar Albornoz,6.8,Junior,Primera A,46,17,3,1,1015,1118,"€450,000",1841,0.7,7.1,11.2,2.5,5.6,1.0,Colombia,South America,1
149,COL,1,RW,75.5,75.9,"M (L), AM (RLC), ST (C)",32,Andrés Ibargüen,7.0,Independiente Medellín,Primera A,77,75,18,7,813,1015,"€475,000",5573,0.7,6.8,8.5,3.1,6.6,1.7,Colombia,South America,1
188,FRA,1,RW,75.2,80.4,"M (LC), AM (RLC)",25,Alexis Claude-Maurice,6.8,Nice,Ligue 1,58,3,2,5,13,14,"€41,500",1625,0.7,7.8,9.7,3.7,5.7,1.4,France,Europe,1
213,BOL,1,ST,75.0,77.4,ST (C),26,Bruno Miranda,6.9,Bolívar,División Profesional,33,26,13,2,0,0,"€240,000",2160,0.7,5.0,8.7,2.1,4.5,2.0,Bolivia,South America,1
236,FRA,1,LB,74.9,76.5,D/WB (L),24,Théo Ndicka Matam,6.7,Grasshoppers,Super League,80,76,1,5,0,0,"€750,000",5595,0.7,9.3,10.8,5.3,6.9,0.2,Switzerland,Europe,1
238,ESP,1,RW,74.9,79.3,"M (C), AM (LC)",28,Cristian Rodríguez,7.0,Castellón,LaLiga 2,80,80,13,11,1015,1119,"€220,000",6850,0.7,6.6,8.3,5.1,3.6,1.0,Spain,Europe,2


### Right wingers

In [45]:
pd.set_option('display.max_columns', 30)
rws = scouting[scouting['P1'] == 'RW']
rws.drop('P1', axis=1, inplace=True)
rws.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
16,JPN,1,LW,78.4,80.1,AM (RL),26,Takahiro Kunimoto,7.7,Johor DT,Liga Super,28,27,7,6,0,0,"€250,000",2324,0.8,2.2,2.6,1.7,3.5,1.6,Malaysia,Asia,1
27,JAM,1,LW,77.2,78.6,"AM (C), ST (C)",31,Bobby Decordova-Reid,6.7,Fulham,Premier League,79,20,6,1,1420,1217,€0,2653,0.7,5.5,7.0,2.4,4.5,1.6,England,Europe,1
59,COL,1,LW,76.8,78.0,"AM (R), ST (C)",31,Arley Rodríguez,6.7,Águilas Doradas,Primera A,80,80,17,1,1419,1217,"€300,000",5429,0.7,4.3,6.9,1.8,5.3,1.8,Colombia,South America,1
66,PAR,1,LW,76.6,78.6,AM (C),31,Oscar Romero,7.0,Pendikspor,Süper Lig,74,74,14,6,0,0,"€325,000",6054,0.7,7.3,8.9,6.2,5.0,1.1,Turkey,Europe,1
90,URU,1,LW,76.4,80.0,AM (R),30,Alejandro Tabó,6.8,Cruz Azul,Liga MX,70,23,3,5,0,0,"€375,000",2443,0.7,6.0,8.4,3.1,6.2,1.0,Mexico,North America,1
128,BUL,1,LW,75.7,76.3,AM (R),31,Birsent Karagaren,7.2,CSKA 1948,Prva liga,24,24,6,2,0,0,"€775,000",1890,0.7,4.9,6.7,3.0,5.8,1.5,Bulgaria,Europe,1
129,COL,1,AM,75.7,78.4,"M (R), AM (RC)",30,Kevin Londoño,7.0,Independiente Medellín,Primera A,61,61,6,15,1116,813,"€550,000",5102,0.7,6.6,9.7,4.1,5.9,0.9,Colombia,South America,1
131,COL,1,LW,75.7,77.6,"M (R), AM (RL)",29,Fredy Salazar,6.7,Águilas Doradas,Primera A,71,18,4,3,1118,915,"€275,000",2475,0.7,5.3,8.8,3.1,5.8,1.7,Colombia,South America,1
132,IRL,1,LW,75.7,79.5,M/AM (C),26,Callum O'Hare,6.8,Coventry,EFL Championship,97,96,10,8,1319,1015,"€575,000",7873,0.7,6.5,8.1,4.3,3.7,0.7,England,Europe,2
138,COL,1,LW,75.6,76.4,AM (R),28,Edwar López,7.1,Deportivo Pasto,Primera A,54,54,12,2,1117,816,"€59,000",4662,0.7,7.4,11.3,3.0,6.7,1.0,Colombia,South America,1


### Strikers

In [46]:
pd.set_option('display.max_columns', 30)
sts = scouting[scouting['P1'] == 'ST']
sts.drop('P1', axis=1, inplace=True)
sts.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
178,GEO,1,RB,75.2,75.0,ST (C),30,Giorgi Kvilitaia,6.9,APOEL,A' Katigorías,55,55,31,1,0,0,"€48,000",4240,0.7,4.0,10.0,1.6,4.8,3.0,Cyprus,Europe,1
299,TUR,1,LW,74.4,74.2,"AM (L), ST (C)",32,Cenk Tosun,6.9,Beşiktaş,Süper Lig,58,44,10,4,914,510,"€155,000",3361,0.7,5.5,9.1,2.2,4.2,2.1,Turkey,Europe,1
496,UKR,1,AM,73.6,74.4,ST (C),28,Artem Besedin,7.0,Ordabasy,Premer Lïgası,35,35,22,3,0,0,"€625,000",2854,0.7,4.3,11.0,1.4,4.4,3.0,Kazakhstan,Europe,1
623,UZB,1,LW,73.2,76.0,ST (C),26,Jasur Yaxshiboyev,7.2,Navbahor,PFL,34,34,9,8,0,0,"€425,000",2807,0.7,6.7,8.7,4.2,7.6,1.8,Uzbekistan,Asia,1
824,MAR,1,RW,72.8,74.2,ST (C),28,Oualid Azaro,6.9,Ajman,UAE Pro League,61,61,28,2,1318,1220,"€750,000",5051,0.7,4.4,12.3,1.4,5.5,2.5,UAE,Asia,1
871,ESP,1,RB,72.6,76.8,ST (C),24,Borja Garcés,7.3,Atlético B,2ª RFEF V,41,32,21,10,1119,1117,"€170,000",3143,0.7,0.3,0.8,0.2,1.6,2.5,Spain,Europe,4
891,ESP,1,LW,72.6,75.9,"AM (L), ST (C)",25,Rafa Mujica,6.7,Arouca,Liga bwin,59,36,9,1,1016,1014,"€725,000",3253,0.7,5.0,10.5,1.7,4.5,2.1,Portugal,Europe,1
919,COL,1,AM,72.5,73.8,"AM (C), ST (C)",31,Jhonatan Agudelo,6.9,Cúcuta Deportivo,Primera B,31,31,13,0,1015,712,"€39,500",2260,0.7,4.5,11.3,1.6,5.3,3.5,Colombia,South America,3
958,URU,1,RW,72.4,73.9,ST (C),30,Adrián Balboa,6.8,Deportivo Pereira,Primera A,78,71,26,8,1118,1117,"€275,000",6134,0.7,4.6,9.2,1.6,5.8,2.2,Colombia,South America,1
1107,URU,1,RW,72.2,73.2,ST (C),28,Maicol Cabrera,6.8,Liverpool F.C.,1ª División,34,27,8,4,0,0,"€110,000",1930,0.7,5.0,10.7,2.0,5.9,2.5,Uruguay,South America,1


## Top N players by continent

### Africa

In [47]:
pd.set_option('display.max_columns', 30)
africa = scouting[scouting['Continent'] == 'Africa']
africa.drop('Continent', axis=1, inplace=True)
africa.head(n_af).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P1,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Lv
23,COM,1,GK,CB,77.4,78.2,GK,32,Ali Ahamada,6.9,Azam FC,Vodacom Premier League,47,47,0,0,0,0,"€81,000",3980,0.3,0.0,0.0,0.1,0.8,0.0,Tanzania,1
67,ALG,1,CB,DM,76.6,78.7,D (C),24,Mohamed Tougai,7.0,ES Tunis,Ligue 1,70,70,3,0,0,0,"€275,000",5933,0.7,6.7,14.5,4.0,1.1,0.4,Tunisia,1
104,ALG,1,LB,CM,76.2,77.0,M (C),30,Zakaria Draoui,7.0,CR Bélouizdad,Ligue 1,64,62,2,13,0,0,"€800,000",4748,0.7,6.5,9.3,5.8,3.4,0.2,Algeria,1
155,ALG,1,RB,CB,75.4,76.9,D (R),31,Houcine Benayada,7.0,CR Bélouizdad,Ligue 1,54,54,3,4,0,0,"€350,000",4318,0.7,9.0,13.2,5.0,7.7,0.6,Algeria,1
166,TUN,1,LB,CB,75.3,77.6,D (L),27,Oussama Shili,7.0,Stade Tunisien,Ligue 1,37,37,2,7,0,0,"€81,000",3038,0.7,9.9,12.6,5.1,6.3,0.2,Tunisia,1
222,MAR,1,DM,CM,75.0,75.7,"DM, M (C)",32,Yahya Jabrane,7.0,Wydad Casablanca,Botola,88,85,4,18,0,0,"€675,000",6959,0.7,7.1,10.5,5.2,2.5,0.4,Morocco,1
224,MAR,1,DM,CM,75.0,77.0,"DM, M (C)",31,Mehdi Oubila,7.1,RS Berkane,Botola,62,59,5,22,0,0,"€11,500",5144,0.7,7.0,8.8,7.2,4.1,0.5,Morocco,1
244,ALG,1,LB,RW,74.8,77.9,D/WB (L),29,Ilyes Chetti,6.9,Wydad Casablanca,Botola,86,85,1,4,0,0,"€650,000",7536,0.7,9.5,11.6,6.2,6.5,0.2,Morocco,1
338,RSA,1,RW,AM,74.2,74.3,AM (RC),32,Vincent Pule,7.1,Orlando Pirates,PSL,67,42,6,12,0,0,"€54,000",3554,0.7,7.2,10.0,4.4,6.9,1.3,South Africa,1
413,COL,1,RW,LW,73.8,75.5,"M/AM (R), ST (C)",31,Luis Hinestroza,7.0,Al-Mokawloon,Premier League,75,73,17,13,1318,1116,"€675,000",5708,0.7,5.2,7.5,2.7,7.3,1.8,Egypt,1


### Europe

In [48]:
pd.set_option('display.max_columns', 30)
europe = scouting[scouting['Continent'] == 'Europe']
europe.drop('Continent', axis=1, inplace=True)
europe.head(n_eu).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P1,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Lv
0,ENG,1,LB,RB,80.9,85.7,D (RL),23,Brandon Williams,6.8,Man Utd,Premier League,68,65,3,5,1318,1320,"€43,000",5750,0.7,9.9,12.9,4.3,7.1,0.4,England,1
8,ITA,1,LB,LW,80.5,85.3,D/WB/M (L),25,Emanuele Valeri,6.8,Cremonese,Serie B,77,77,1,8,1117,816,"€275,000",6452,0.7,8.8,10.6,5.0,7.2,0.3,Italy,2
10,POR,1,CM,DM,80.1,81.1,"DM, M/AM (C)",28,Samu,6.8,Vizela,Liga bwin,78,78,0,10,11,13,"€155,000",6755,0.7,7.8,9.4,5.8,3.1,0.3,Portugal,1
11,ITA,1,LB,RB,79.1,83.1,D/WB (L),24,Gianluca Frabotta,6.9,Juventus,Serie A,42,39,0,5,1217,814,"€675,000",3403,0.7,8.9,12.7,6.8,5.5,0.1,Italy,1
12,CZE,1,LB,CM,79.0,79.6,"D/WB (L), M (C)",32,Martin Frýdek,6.9,Luzern,Super League,65,65,1,4,0,0,"€130,000",5699,0.7,9.1,13.6,6.5,5.7,0.3,Switzerland,1
14,DEN,1,LB,LW,78.8,79.7,D (L),29,Casper Højer,7.0,Sparta Praha,1. ČFL,83,81,1,10,0,0,"€625,000",7121,0.7,8.9,12.3,7.5,6.4,0.1,Czech Republic,1
15,ITA,1,RB,LB,78.5,80.0,D/WB (R),31,Stefano Sabelli,6.8,Genoa,Serie A,69,67,0,4,1420,1217,"€350,000",5189,0.7,8.7,12.5,3.9,6.3,0.2,Italy,1
17,ENG,1,RB,CB,78.2,78.7,D (R),32,Luke Ayling,6.8,Leeds,EFL Championship,38,26,0,4,712,1018,€0,2329,0.7,8.9,13.2,4.4,5.5,0.2,England,2
18,NGA,1,CM,DM,78.0,81.4,M/AM (C),26,Michael Folorunsho,6.8,Napoli,Serie A,41,37,3,3,1117,614,"€27,500",2977,0.7,7.5,11.4,5.9,3.3,0.6,Italy,1
19,NED,1,RB,LB,78.0,78.7,"D (RL), WB/AM (R)",30,Joshua Brenet,7.0,Twente,Eredivisie,104,102,13,7,1218,916,"€525,000",9257,0.7,7.9,11.5,4.9,6.7,0.8,Netherlands,1


### South America

In [49]:
pd.set_option('display.max_columns', 30)
south_am = scouting[scouting['Continent'] == 'South America']
south_am.drop('Continent', axis=1, inplace=True)
south_am.head(n_sa).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P1,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Lv
9,BRA,1,LB,RB,80.3,81.3,D (L),31,Renê,6.8,Internacional,Brasileirão Assaí,43,42,1,2,14,13,"€575,000",3552,0.7,9.0,11.4,6.8,4.9,0.2,Brazil,1
20,ARG,1,LB,RB,77.8,79.5,D (L),25,Facundo Castet,7.0,Sarmiento (Junín),Primera Nacional,48,48,2,2,0,0,"€650,000",4181,0.7,5.2,7.2,4.0,3.9,0.4,Argentina,2
21,ARG,1,CM,DM,77.8,78.8,M (C),30,Leonel Miranda,6.9,Racing Club,Liga Profesional,37,30,1,4,0,0,"€300,000",2503,0.7,6.7,8.4,7.1,3.0,0.3,Argentina,1
22,BOL,1,RB,LB,77.6,78.3,D (R),32,Diego Bejarano,7.0,Bolívar,División Profesional,43,40,0,6,0,0,"€6,500",3523,0.7,8.5,12.4,5.5,6.8,0.3,Bolivia,1
26,COL,1,RB,RW,77.3,81.6,D/WB (R),29,Walmer Pacheco,6.8,Junior,Primera A,72,70,0,4,1217,813,"€400,000",6179,0.7,8.8,11.7,5.1,6.6,0.4,Colombia,1
35,BRA,1,RB,LB,77.2,81.4,"D (RL), WB (R)",29,Zeca,6.9,Vitória,Brasileirão SportingBet Série B,60,60,4,9,14,13,"€115,000",5243,0.7,9.5,12.2,7.3,6.3,0.3,Brazil,2
38,ECU,1,LB,CB,77.1,78.0,D (LC),26,Luis Segovia,6.8,Botafogo,Brasileirão Assaí,20,18,0,0,13,13,"€375,000",1638,0.7,7.0,13.8,4.5,1.4,0.3,Brazil,1
41,ARG,1,RB,LB,77.0,78.1,D/WB (R),27,Hernán de la Fuente,6.9,nan,nan,79,74,3,6,14,14,€0,6379,0.7,9.5,12.3,5.9,7.0,0.4,Argentina,0
50,ARG,1,RB,RW,77.0,78.7,D/WB (R),25,Alex Vigo,6.8,River,Liga Profesional,28,27,1,1,0,0,"€800,000",2481,0.7,9.3,13.1,5.0,5.8,0.5,Argentina,1
56,BRA,1,LB,RB,76.9,77.3,D/WB (L),27,Jefferson,7.0,Atlético Goianiense,Brasileirão SportingBet Série B,56,49,3,10,12,13,"€400,000",4404,0.7,8.8,13.3,7.2,4.6,0.2,Brazil,2


### North America

In [50]:
pd.set_option('display.max_columns', 30)
north_am = scouting[scouting['Continent'] == 'North America']
north_am.drop('Continent', axis=1, inplace=True)
north_am.head(n_na).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P1,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Lv
90,URU,1,RW,LW,76.4,80.0,AM (R),30,Alejandro Tabó,6.8,Cruz Azul,Liga MX,70,23,3,5,0,0,"€375,000",2443,0.7,6.0,8.4,3.1,6.2,1.0,Mexico,1
108,URU,1,RB,CB,76.1,77.2,D (C),30,Yonatthan Rak,6.8,Tijuana,Liga MX,24,23,0,1,0,0,"€350,000",2023,0.7,7.7,14.7,4.3,1.1,0.0,Mexico,1
176,USA,1,LB,RB,75.2,78.6,D (L),27,Chase Gasper,6.8,Houston,MLS,42,33,0,0,0,0,"€400,000",3208,0.7,9.5,13.7,6.5,4.7,0.2,United States,1
255,COL,1,LW,ST,74.7,77.4,AM (L),26,Juan Camilo Salazar,7.6,FAS,La Liga Mayor,45,45,11,16,0,0,"€87,000",3733,0.8,6.6,8.6,3.4,6.9,1.6,El Salvador,1
260,MEX,1,DM,CM,74.7,79.8,"DM, M (C)",28,Andrés Iniestra,6.8,At. San Luis,Liga MX,44,15,0,1,0,0,"€475,000",1918,0.7,6.1,8.9,6.7,2.5,0.3,Mexico,1
263,COL,1,RW,ST,74.7,74.8,ST (C),32,Edis Ibargüen,6.9,Marathón,Liga Nacional Honduras,49,49,17,6,0,0,"€40,000",4057,0.7,3.9,8.7,1.8,6.2,2.0,Honduras,1
266,URU,1,LB,CB,74.6,77.7,D (C),26,Enzo Martínez,6.7,Pachuca,Liga MX,34,25,2,0,0,0,"€350,000",2660,0.7,7.2,12.7,5.1,2.1,0.2,Mexico,1
287,ARG,1,CB,RB,74.4,74.8,D (C),32,Leandro González Pírez,6.8,Inter Miami,MLS,36,23,1,1,0,0,"€375,000",2272,0.7,6.7,14.0,3.8,1.3,0.2,United States,1
342,MEX,1,LB,DM,74.2,79.0,D (L),26,Aldo Cruz,6.8,FC Juárez,Liga MX,74,73,0,2,0,0,"€110,000",5962,0.7,10.2,12.4,6.8,4.7,0.1,Mexico,1
353,ESP,1,RB,RW,74.1,80.2,D/WB (R),28,Marc Navarro,7.7,El Paso,USLC Western Conference,38,37,4,2,0,0,"€600,000",3336,0.8,0.2,0.3,0.4,1.6,0.4,United States,2


### Asia

In [51]:
pd.set_option('display.max_columns', 30)
asia = scouting[scouting['Continent'] == 'Asia']
asia.drop('Continent', axis=1, inplace=True)
asia.head(n_as).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P1,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Lv
16,JPN,1,RW,LW,78.4,80.1,AM (RL),26,Takahiro Kunimoto,7.7,Johor DT,Liga Super,28,27,7,6,0,0,"€250,000",2324,0.8,2.2,2.6,1.7,3.5,1.6,Malaysia,1
24,ARG,1,DM,CM,77.4,79.0,"DM, M (C)",31,Gerónimo Poblete,6.9,Al-Wasl,UAE Pro League,50,50,0,11,1018,1016,"€775,000",4477,0.7,7.3,10.2,7.0,3.1,0.2,UAE,1
62,IRQ,1,LB,LW,76.7,77.2,D/WB/M (L),30,Dhurgham Ismail,7.0,Al-Talaba,Iraqi Premier League,43,43,1,5,0,0,"€43,000",3827,0.7,9.1,11.3,7.0,6.8,0.3,Iraq,1
93,IRQ,1,LB,LW,76.3,79.5,D/WB (L),30,Ali Adnan,6.8,Mes Rafsanjan,Persian Gulf League,59,59,1,3,0,0,"€650,000",5118,0.7,8.1,11.6,4.2,6.6,0.2,Iran,1
125,JPN,1,LB,RB,75.8,78.1,D/WB (L),27,Ryôya Ogawa,7.1,FC Tokyo,J1 League,61,61,1,7,0,0,"€675,000",5338,0.7,4.4,6.1,3.8,3.3,0.2,Japan,1
145,ALG,1,DM,CM,75.6,77.2,"DM, M (C)",30,Victor Lekhal,6.8,Umm-Salal,QSL,33,33,0,5,714,1217,"€800,000",2527,0.7,7.6,11.5,6.5,3.0,0.3,Qatar,1
152,PER,1,RW,LW,75.4,76.2,AM (RL),32,André Carrillo,6.9,Al-Qadsiah,Saudi League,45,45,2,7,1220,814,"€500,000",3667,0.7,6.6,8.4,6.6,5.4,0.6,Saudi Arabia,1
165,SWE,1,GK,CB,75.4,76.8,GK,30,Jacob Rinne,6.8,Al-Fateh,Saudi League,29,29,0,0,0,0,"€170,000",2574,0.3,0.0,0.0,0.0,0.5,0.0,Saudi Arabia,1
174,BRA,1,CM,DM,75.2,75.9,M (C),32,Éber Bessa,7.1,Bali United,Liga 1,54,54,14,17,0,0,€0,4637,0.7,7.0,8.6,4.8,4.8,1.0,Indonesia,1
193,TOG,1,RW,RB,75.1,78.4,M/AM (R),29,Khaled Narey,7.1,Al-Khaleej (KSA),Saudi D1,54,53,7,7,915,1318,"€67,000",4667,0.7,7.8,11.0,4.7,6.6,1.0,Saudi Arabia,2


### Oceania

In [52]:
pd.set_option('display.max_columns', 30)
oceania = scouting[scouting['Continent'] == 'Oceania']
oceania.drop('Continent', axis=1, inplace=True)
oceania.head(n_oc).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P1,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Lv
2453,NZL,1,LB,CB,70.5,71.9,D (L),29,Francis de Vries,6.9,Eastern Suburbs,Northern League,42,42,0,3,0,0,€0,3645,0.7,8.3,14.3,5.6,3.4,0.2,New Zealand,1
7221,NZL,1,AM,RW,66.6,67.6,"AM (C), ST (C)",31,Garbhan Coughlan,6.6,Cashmere Tech,Southern League,38,38,9,3,0,0,€0,3325,0.7,4.5,9.0,2.3,5.5,1.6,New Zealand,1
7418,MWI,1,RB,LB,66.5,70.6,D (R),24,Kelvin Kalua,6.8,Eastern Suburbs,Northern League,42,42,0,4,0,0,"€34,000",3714,0.7,9.2,14.6,5.1,6.5,0.2,New Zealand,1
7604,NZL,1,RB,LB,66.4,67.5,D (RC),31,Justin Gulley,6.8,Wellington Olympic,Central League,37,37,6,2,0,0,€0,3330,0.7,8.1,17.9,5.1,1.0,0.9,New Zealand,1
7749,NZL,1,RW,RB,66.3,70.0,WB/M (R),26,Jack-Henry Sinclair,6.8,Wellington Olympic,Central League,37,37,2,3,0,0,€0,3243,0.7,8.5,11.2,3.9,7.4,0.3,New Zealand,1


## Top N players by metric

### GK% - Goalkeeping prowess

In [53]:
pd.set_option('display.max_columns', 30)
metric_gk = scouting[(scouting['P1'] == 'GK') &
                     scouting['Continent'].isin(continents_metrics) &
                     (scouting['Lv'] <= max_level)]
metric_gk = metric_gk.sort_values('GK', ascending=False)
metric_gk.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P1,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
6612,POL,1,GK,CB,67.1,68.1,GK,31,Konrad Forenc,6.9,Korona,1. Liga,34,34,0,0,0,0,"€27,000",2966,1.5,0.0,0.0,0.1,0.6,0.0,Poland,Europe,2
5572,ECU,1,GK,RB,67.8,70.5,GK,31,Johan Padilla,7.4,Macará,Serie B de Ecuador,24,24,0,0,0,0,"€46,000",2051,1.5,0.0,0.0,0.0,0.6,0.0,Ecuador,South America,2
2664,ROU,1,GK,RB,70.3,74.3,GK,25,Andrei Vlad,6.9,FCSB,Liga I,27,22,0,0,0,0,"€500,000",2146,1.4,0.0,0.0,0.0,0.7,0.0,Romania,Europe,1
4937,ITA,1,GK,CB,68.3,69.4,GK,28,Marco Bleve,7.0,Lecce,Serie A,48,47,0,0,0,0,"€175,000",4277,1.2,0.0,0.0,0.0,0.6,0.0,Italy,Europe,1
4068,ITA,1,GK,CB,69.0,71.2,GK,29,Lorenzo Andrenacci,6.9,Brescia,Serie B,39,26,0,0,0,0,"€235,000",2939,1.1,0.0,0.0,0.0,0.5,0.0,Italy,Europe,2
15677,NOR,1,GK,CB,61.0,64.7,GK,24,Simen Vidtun Nilsen,6.7,Sarpsborg,Eliteserien,20,19,0,0,0,0,"€21,500",1569,1.1,0.0,0.0,0.0,0.6,0.0,Norway,Europe,1
11570,POL,1,GK,CB,63.6,64.6,GK,31,Mateusz Abramowicz,7.0,Miedź,1. Liga,27,27,0,0,0,0,"€80,000",2344,1.0,0.0,0.0,0.3,0.5,0.0,Poland,Europe,2
10394,AUT,1,GK,CB,64.4,65.6,GK,28,Johannes Kreidl,6.9,KuPS,Veikkausliiga,30,30,0,0,0,0,"€775,000",2412,1.0,0.0,0.0,0.1,0.6,0.0,Finland,Europe,1
26093,CRO,1,GK,CB,52.9,61.0,GK,19,Luigi Mišević,6.8,Šibenik,1. NL,28,23,0,0,0,0,"€37,500",2174,1.0,0.0,0.0,0.1,0.4,0.0,Croatia,Europe,2
8743,KAZ,1,GK,CB,65.6,69.9,GK,25,Bekkhan Shaizada,7.1,Ordabasy,Premer Lïgası,38,38,0,0,0,0,"€600,000",3262,1.0,0.0,0.0,0.0,0.5,0.0,Kazakhstan,Europe,1


### DFg% - Defensive ground actions

In [54]:
pd.set_option('display.max_columns', 30)
positions_metric = ['CB', 'LB', 'RB', 'DM']  
metric_dfg = scouting[(scouting['P1'].isin(positions_metric)) &
                      scouting['Continent'].isin(continents_metrics) &
                      (scouting['Lv'] <= max_level)]

metric_dfg = metric_dfg.sort_values('DFg', ascending=False)
metric_dfg.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P1,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
7201,POL,1,RB,RW,66.6,71.2,D/WB (R),22,Bartłomiej Kłudka,7.3,KGHM Zagłębie,2. Liga,34,33,5,5,0,0,"€300,000",2860,0.7,11.6,17.8,4.7,6.2,0.9,Poland,Europe,2
17663,TUR,1,RB,RW,59.7,65.5,D (R),23,Talha Ülvan,6.4,Giresunspor,1. Lig,56,37,2,0,0,0,"€7,750",3306,0.6,11.4,15.9,4.4,5.2,0.4,Turkey,Europe,2
18827,PAR,1,LB,LW,59.1,62.2,M (L),28,Diego Rojas,6.6,Sol de América,Primera División,28,23,1,1,0,0,"€3,300",1809,0.7,11.3,14.6,5.6,4.4,0.2,Paraguay,South America,1
14228,IRL,1,RB,LB,61.8,66.0,D/WB (RL),23,Aaron McNally,6.8,Drogheda Utd,League of Ireland Premier Division,30,30,1,1,0,0,"€300,000",2572,0.7,11.3,15.2,4.2,4.7,0.3,Ireland,Europe,1
13243,EST,1,LB,LW,62.4,68.3,D/AM (L),22,Michael Schjønning-Larsen,6.9,FCI Levadia,Premium Liiga,21,21,4,3,0,0,"€79,000",1851,0.7,11.3,14.4,4.6,4.5,1.4,Estonia,Europe,1
6803,CHI,1,RB,CB,67.0,74.3,D (RC),26,Brian Torrealba,6.9,O'Higgins,Primera División,39,37,3,1,0,0,"€135,000",3051,0.7,11.1,16.7,4.7,4.0,0.6,Chile,South America,1
8002,GER,1,RB,LB,66.1,68.4,D (R),31,Marko Karamarko,6.8,Velež Mostar,Premijer Liga BiH,42,42,2,0,0,0,"€71,000",3546,0.7,11.1,16.6,4.7,4.9,0.5,Bosnia & Herzegovina,Europe,1
16670,ALB,1,RB,RW,60.3,66.7,D (R),25,Anteo Osmanllari,6.8,KF Erzeni,Kategoria Superiore,45,45,1,1,0,0,"€52,000",4050,0.7,11.0,14.4,4.6,4.9,0.5,Albania,Europe,1
5215,ECU,1,RB,RW,68.1,73.1,D (R),23,Daykol Romero,6.7,Liga de Quito,Serie A de Ecuador,53,52,3,4,0,0,"€110,000",4368,0.7,10.9,14.0,5.0,4.9,0.4,Ecuador,South America,1
14347,ALB,1,RB,RW,61.8,66.0,D (R),21,Reild Kurti,6.6,KF Teuta,Kategoria Superiore,48,48,3,2,0,0,"€105,000",4222,0.7,10.9,13.1,4.1,4.5,0.7,Albania,Europe,1


### DFa% - Defensive actions including aerial

In [55]:
pd.set_option('display.max_columns', 30)
positions_metric = ['CB', 'DM']  
metric_dfa = scouting[(scouting['P1'].isin(positions_metric)) &
                      scouting['Continent'].isin(continents_metrics) &
                      (scouting['Lv'] <= max_level)]

metric_dfa = metric_dfa.sort_values('DFa', ascending=False)
metric_dfa.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P1,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
10020,SWE,1,CB,RW,64.7,73.6,D (C),23,Teodor Stenshagen,6.5,GIF Sundsvall,Allsvenskan,20,17,0,0,0,0,"€135,000",1576,0.7,8.2,21.8,3.7,0.7,0.2,Sweden,Europe,1
18719,IRL,1,CB,RB,59.1,60.5,D (C),26,Conor Keeley,6.5,Drogheda Utd,League of Ireland Premier Division,36,36,1,1,0,0,"€110,000",3216,0.7,8.1,21.5,4.5,0.6,0.2,Ireland,Europe,1
23930,BOL,1,CB,RB,55.6,62.5,D (C),25,Leonardo Urapuca,6.5,Universitario de Sucre,División Profesional,49,49,0,2,0,0,"€21,500",4297,0.7,8.4,21.5,5.2,0.6,0.1,Bolivia,South America,1
20923,POL,1,CB,RB,57.8,63.8,D (C),23,Kacper Józefiak,6.7,Miedź,1. Liga,29,20,0,0,0,0,"€55,000",1998,0.7,8.8,21.4,5.1,0.8,0.3,Poland,Europe,2
14057,SCO,1,CB,LB,61.9,61.9,D (C),29,Ally Gilchrist,6.9,Cork City,League of Ireland Premier Division,35,35,2,1,0,0,"€325,000",3150,0.7,8.5,21.4,5.0,0.7,0.3,Ireland,Europe,1
9287,POL,1,CB,DM,65.2,67.1,D (C),30,Krystian Nowak,6.8,Bohemians,League of Ireland Premier Division,33,33,1,2,0,0,"€400,000",2917,0.7,7.8,21.4,4.5,1.0,0.4,Ireland,Europe,1
14698,RUS,1,CB,DM,61.6,62.4,D (C),30,Yegor Potapov,6.7,Maqtaaral,Premer Lïgası,37,37,4,0,0,0,"€20,500",3037,0.7,8.4,21.3,4.8,0.7,0.7,Kazakhstan,Europe,1
21543,LTU,1,CB,RB,57.4,58.4,D (C),29,Tomas Rapalavicius,6.7,Džiugas,A Lyga,25,25,3,0,0,0,"€6,000",2250,0.7,8.7,21.1,4.8,0.6,0.5,Lithuania,Europe,1
9143,ENG,1,CB,RB,65.3,69.5,D (C),25,Jack Sanders,6.7,Kilmarnock,cinch Premiership,48,40,5,1,0,0,€0,3886,0.7,8.7,21.1,4.2,0.7,0.7,Scotland,Europe,1
14760,CMR,1,CB,RB,61.5,62.8,D (C),24,Thierry Etongou,6.7,Radnički,SuperLiga,61,59,2,1,0,0,"€110,000",5270,0.7,8.1,20.8,4.7,0.9,0.3,Serbia,Europe,1


### PAS% - Passing chance creation

In [56]:
pd.set_option('display.max_columns', 30)
positions_metric = ['DM', 'CM', 'AM', 'LW', 'RW', 'ST']  
metric_pas = scouting[(scouting['P1'].isin(positions_metric)) &
                      scouting['Continent'].isin(continents_metrics) &
                      (scouting['Lv'] <= max_level)]

metric_pas = metric_pas.sort_values('PAS', ascending=False)
metric_pas.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P1,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
439,URU,1,LW,LB,73.7,76.7,D/WB/AM (L),23,Kevin Rolón,6.9,Wanderers,1ª División,38,36,1,8,0,0,"€275,000",3181,0.7,8.7,10.9,7.9,5.8,0.2,Uruguay,South America,1
5137,URU,1,RW,LW,68.1,72.2,AM (R),30,Renato César,7.1,Guayaquil City,Serie A de Ecuador,36,33,4,10,0,0,"€525,000",2205,0.7,6.9,8.8,7.8,5.6,0.6,Ecuador,South America,1
5607,ECU,1,DM,CM,67.8,70.2,M (C),30,Richard Calderón,6.9,Orense,Serie A de Ecuador,29,23,0,8,0,0,"€400,000",2016,0.7,6.8,9.1,7.8,2.5,0.3,Ecuador,South America,1
3060,BIH,1,DM,CM,69.9,75.8,DM,26,Petar Bojo,6.8,Zrinjski,Premijer Liga BiH,45,35,1,15,0,0,"€235,000",3050,0.7,7.2,8.5,7.7,2.2,0.4,Bosnia & Herzegovina,Europe,1
1422,BRA,1,RW,LW,71.7,75.0,"M (R), AM (RL)",24,Cristian Dal Bello,7.1,Mladost GAT,SuperLiga,54,54,8,9,0,0,"€325,000",4625,0.7,7.5,9.8,7.7,5.2,0.8,Serbia,Europe,1
1076,PER,1,RW,LW,72.2,72.5,AM (R),28,Alexis Rojas,6.9,Sport Huancayo,Liga 1,44,27,0,4,0,0,"€155,000",2501,0.7,8.1,9.2,7.7,7.3,0.5,Peru,South America,1
771,BOL,1,DM,CM,72.9,74.4,M (C),31,Leonel Justiniano,6.9,Bolívar,División Profesional,64,58,2,13,0,0,"€86,000",4681,0.7,7.4,9.9,7.6,2.5,0.3,Bolivia,South America,1
4828,CMR,1,LW,RW,68.4,73.0,AM (L),27,Alexis Alégué,6.9,Jablonec,1. ČFL,62,58,1,7,0,0,"€450,000",5208,0.7,9.0,12.4,7.6,5.1,0.1,Czech Republic,Europe,1
186,COL,1,DM,CM,75.2,77.7,"DM, M (C)",27,Bryan Rovira,6.9,Univ. Católica,Primera División,37,36,2,7,0,0,"€16,500",2987,0.7,7.3,10.8,7.5,3.0,0.5,Chile,South America,1
7714,SVN,1,DM,CM,66.3,67.1,"DM, M (C)",27,Jon Šporn,6.7,Chornomorets,UPL,53,52,1,4,0,0,"€325,000",3956,0.7,6.8,8.6,7.5,2.3,0.2,Ukraine,Europe,1


### DRB% - Dribbling capabilities

In [57]:
pd.set_option('display.max_columns', 30)
positions_metric = ['DM', 'CM', 'AM', 'LW', 'RW', 'ST']  
metric_drb = scouting[(scouting['P1'].isin(positions_metric)) &
                      scouting['Continent'].isin(continents_metrics) &
                      (scouting['Lv'] <= max_level)]

metric_drb = metric_drb.sort_values('DRB', ascending=False)
metric_drb.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P1,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
23958,ALB,1,LW,RW,55.6,65.4,AM (LC),24,Serkan Basha,7.0,US Hostert,Nationaldivisioun,39,38,3,9,0,0,"€16,750",2583,0.7,8.1,9.4,4.0,8.7,0.8,Luxembourg,Europe,1
7472,POR,1,RW,LW,66.5,70.5,AM (R),30,Frédéric Maciel,7.1,Oţelul Galaţi,Liga II,61,57,6,9,0,0,"€20,000",4005,0.7,6.3,9.7,3.6,8.5,0.9,Romania,Europe,2
6955,CZE,1,LW,RW,66.9,76.6,AM (L),21,Jáchym Šíp,7.1,Sigma Olomouc,2. ČFL,36,36,6,5,0,0,"€625,000",2886,0.7,7.0,9.0,4.1,8.3,0.9,Czech Republic,Europe,2
8690,CGO,1,LW,RW,65.6,66.8,ST (C),21,Beni Makouana,6.8,Polissia,UPL,27,27,4,3,0,0,"€650,000",2383,0.7,6.8,9.4,2.7,8.2,1.0,Ukraine,Europe,1
10887,CIV,1,RW,LW,64.1,66.8,AM (RL),22,Sosthène Zilé,7.0,Ararat,Premiyer Liga,41,41,4,6,0,0,"€240,000",3631,0.7,6.0,7.9,3.1,8.2,1.2,Armenia,Europe,1
5191,NIR,1,RW,LW,68.1,73.2,AM (R),29,Matty Kennedy,6.7,Kilmarnock,cinch Premiership,72,63,1,7,0,0,"€84,000",5496,0.7,8.4,10.4,3.8,8.1,0.2,Scotland,Europe,1
1119,BEL,1,RW,LW,72.1,76.5,M/AM (RL),24,Thibaud Verlinden,7.1,Beerschot,Eerste klasse B,53,49,3,13,1016,815,"€775,000",3501,0.7,7.4,8.9,6.1,8.1,0.7,Belgium,Europe,2
4886,VEN,1,RW,LW,68.3,70.1,M/AM (RL),29,Yanoswky Reyes,7.1,Zamora F.C. (VEN),Liga FUTVE,50,50,10,9,916,1217,"€79,000",4169,0.7,6.1,7.7,3.0,8.1,1.2,Venezuela,South America,1
5367,SRB,1,LW,RW,68.0,68.7,AM (L),32,Lazar Arsić,6.8,Mladost GAT,SuperLiga,43,17,2,4,0,0,"€90,000",1769,0.7,7.7,9.7,4.3,8.0,0.7,Serbia,Europe,1
8314,UKR,1,LW,RW,65.9,70.6,AM (L),25,Andriy Remeniuk,6.8,Metalist 1925,UPL,35,33,2,2,0,0,"€175,000",2069,0.7,7.8,10.3,2.8,8.0,1.0,Ukraine,Europe,1


### ST% - Striking threat

In [58]:
pd.set_option('display.max_columns', 30)
positions_metric = ['DM', 'CM', 'AM', 'LW', 'RW', 'ST']  
metric_st = scouting[(scouting['P1'].isin(positions_metric)) &
                     scouting['Continent'].isin(continents_metrics) &
                     (scouting['Lv'] <= max_level)]

metric_st = metric_st.sort_values('ST', ascending=False)
metric_st.head(n_position).style.background_gradient(cmap='RdYlGn', vmin=0).set_properties(**{'font-size': '14px'}).format(precision=1)

,Nat,Int,P1,P2,RtG,PtG,Position,Age,Name,AvR,Club,Division,App,Str,Gls,Ast,Acc,Pac,Value,Mins,GK,DFg,DFa,PAS,DRB,ST,Based,Continent,Lv
26342,NGA,1,ST,RB,52.4,61.1,ST (C),25,Tony Obonogwu,7.1,Marsaxlokk,BOV Premier League,35,28,25,2,0,0,"€13,500",2534,0.7,3.7,11.3,2.1,4.1,3.9,Malta,Europe,1
14333,BIH,1,ST,RB,61.8,63.4,ST (C),24,Aleksa Mrđa,6.9,Jedinstvo (BP),1. CFL,41,13,15,0,0,0,"€125,000",1669,0.7,3.9,14.0,1.3,5.1,3.7,Montenegro,Europe,1
20562,ESP,1,RW,ST,58.0,66.3,ST (C),23,Jalen Miller Blesa,6.7,Prishtina,Superliga,47,8,10,2,0,0,"€40,500",1530,0.7,4.4,7.7,1.6,4.4,3.7,Kosovo,Europe,1
1779,ITA,1,LW,RW,71.2,75.2,"AM (L), ST (C)",27,Luca Vido,6.8,Reggiana,Serie B,58,11,13,1,1116,713,"€675,000",1751,0.7,3.9,6.9,1.4,5.4,3.7,Italy,Europe,2
3082,PER,1,RW,ST,69.8,73.8,ST (C),29,Aldair Rodríguez,6.8,Alianza,Liga 1,62,21,14,1,0,0,"€160,000",2394,0.7,5.0,11.6,1.6,4.9,3.6,Peru,South America,1
2344,BRA,1,LW,ST,70.6,74.8,ST (C),29,Francisco da Costa,6.9,Bolívar,División Profesional,41,17,11,1,0,0,"€210,000",1679,0.7,4.2,12.1,1.0,4.9,3.6,Bolivia,South America,1
1411,BRA,1,ST,RB,71.7,73.7,ST (C),27,Diogo,7.0,Plaza Colonia,1ª División,56,56,38,1,0,0,"€210,000",4776,0.7,4.3,10.9,1.5,4.7,3.6,Uruguay,South America,1
7694,VEN,1,ST,RB,66.3,71.2,ST (C),23,Bryan Castillo,6.8,Deportivo Táchira,Liga FUTVE,55,22,14,2,1015,612,"€130,000",2330,0.7,4.8,14.3,1.6,3.8,3.5,Venezuela,South America,1
4360,COL,1,ST,RW,68.8,74.2,ST (C),23,Brayan León,6.8,Independiente Medellín,Primera A,70,14,19,2,914,1118,"€475,000",2429,0.7,4.4,7.6,1.1,4.4,3.5,Colombia,South America,1
254,NOR,1,RW,LW,74.7,75.8,ST (C),31,Fredrik Gulbrandsen,6.7,Molde,Eliteserien,53,17,9,1,0,0,"€425,000",1988,0.7,4.0,7.1,1.7,4.5,3.5,Norway,Europe,1
